# Building Complex Contact Schema

One of the cool things of the new desing of ProLint is that you can build very complex contact representations. Similar to building blocks. We showcase a few examples here which we think are useful and interesting.

In [14]:
from prolint2 import Universe

In [15]:
u = Universe('coordinates.gro', 'trajectory.xtc')
u.normalize_by = 'total_time' # We will use true time normalization, though any of the available options will work just as well

In [16]:
### Compute contacts at 3 different cutoffs

In [17]:
c1 = u.compute_contacts(cutoff=6)
c2 = u.compute_contacts(cutoff=7)
c3 = u.compute_contacts(cutoff=8)

100%|██████████| 179/179 [00:01<00:00, 159.71it/s]


### Multiple Cutoffs

If you would like to use a `Double cutoff` as a damping layer, which will allow you to measure contact durations more accurately, then this can be very easily done with a very intuitive syntax:

In [19]:
c3 = c1.intersection(c2)

In [24]:
# Let's again loog at residue 401 with CHOL
r401_chol = sorted(c3.contacts[401]['CHOL'], reverse=True)
print([round(x, 3) for x in r401_chol])

[0.168, 0.14, 0.14, 0.14, 0.112, 0.084, 0.084, 0.084, 0.084, 0.084, 0.084, 0.056, 0.056, 0.056, 0.056, 0.056, 0.056, 0.056, 0.056, 0.056, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028]


### Why this works

Above we are computing the intersection between `c1` and `c2`, and that is all there is to it. The reasoning behind this is quite simple. Cutoffs are radial, which in our case, has the physical interpretation of $c1 \subseteq c2$, That is, the contacts with the smaller cutoff are a subset of the contacts with the larger cutoff. This is a very intuitive way of thinking about it, and it allows us to build very complex contact representations with ease, by noting the following: 

If `c1` and `c2` are sets of numbers, and `c1` is a subset of `c2` (i.e., $c1 \subseteq c2$), this relationship can be reflected in the following common set operations:

1. Union ($c1 \cup c2$): Since $c1$ is a subset of $c2$, the union of $c1$ and $c2$ will be equal to $c2$. This is because all elements of $c1$ are already present in $c2$, so the union will not add any new elements to the resulting set.

2. Intersection ($c1 \cap c2$): The intersection of $c1$ and $c2$ will be equal to $c1$. This is because all elements of $c1$ are present in $c2$, so the intersection will include all elements of $c1$.

3. Difference ($c2 - c1$): The difference of $c2$ and $c1$ will be the set of elements that are present in $c2$ but not in $c1$. Since $c1$ is a subset of $c2$, the difference will include all the elements unique to $c2$.

4. Difference ($c1 - c2$): The difference of $c1$ and $c2$ will be the empty set ($\emptyset$), since all elements of $c1$ are present in $c2$. There are no elements unique to $c1$ that are not in $c2$.

5. Symmetric Difference ($c1 \bigtriangleup c2$): The symmetric difference of $c1$ and $c2$ will also be equal to the difference ($c2 - c1$). This is because symmetric difference includes elements that are unique to one of the sets, and since $c1$ is a subset of $c2$, all unique elements are in $c2$ but not in $c1$.

In summary, for sets $c1$ and $c2$ where $c1 \subseteq c2$:

- $c1 \cup c2 = c2$
- $c1 \cap c2 = c1$
- $c2 - c1 = \text{elements unique to } c2$
- $c1 - c2 = \emptyset$
- $c1 \bigtriangleup c2 = c2 - c1$


In practice this results in a trivially easy way to construct complex contact schemas: 

In [ ]:
# Compute more contacts at different cutoffs
c3 = u.compute_contacts(cutoff=8)
c4 = u.compute_contacts(cutoff=9)
c5 = u.compute_contacts(cutoff=10)

In [ ]:
_ = c1 + c2                 # Double cutoff 
_ = c1 + c2 + c3            # Triple cutoff
_ = c1 + c2 + c3 + c4       # Quadruple cutoff
_ = c1 + c2 + c3 + c4 + c5  # Quintuple cutoff

This way you can, for example, look at how contacts with different lipids fall of as a function of the cutoff

### Annular Shell Contact

We can also go in the other direction, and construct a donut shape contact schema, where we have a core of contacts with a smaller cutoff, and a shell of contacts with a larger cutoff, and we substract/remove the core from the shell:

In [25]:
c2 - c1 # or c2.difference(c1)

Such an analysis can be very useful when studying lipid-protein interactions where the concept of annular lipids is very important. With the logic and syntax developed here, we can very easily construct a contact schema that will allow us to study the annular lipids as a function of the cutoff.

In [ ]:
first_shell  = c2 - c1 # or c2.difference(c1)
second_shell = c3 - c2 # or c3.difference(c2)
third_shell  = c4 - c3 # or c4.difference(c3)
fourth_shell = c5 - c4 # or c5.difference(c4)